In [1]:
import pandas as pd
import numpy as np
import requests
import os
# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
DATIUM_DATASET_URL = 'https://drive.google.com/uc?export=download&id=1kXe7p6o6IDr9KwfOG2vPk8TEDsnMukAt'
DATIUM_DATASET_PATH = '../datasets/DatiumSample.rpt'
OUTPUT_DATASET_PATH = '../datasets/dataset_1.csv'
DROP_COLS = ['AvgWholesale', 'AvgRetail', 'GoodWholesale', 'GoodRetail', 'TradeMin', 'TradeMax', 'PrivateMax']
TARGET_COL = 'Sold_Amount'
CUTOFF_NULL_PCT = 25 # Maximum % of Nulls Before Dropping Column

In [3]:
if not os.path.exists(DATIUM_DATASET_PATH):
    print(f'INFO - Downloading Datium Dataset to {DATIUM_DATASET_PATH}')
    response=requests.get(DATIUM_DATASET_URL)
    with open('../datasets/DatiumSample.rpt', 'wb')as file:
        file.write(response.content)
else:
    print(f'INFO - Skip Download. Datium Dataset found at {DATIUM_DATASET_PATH}')

INFO - Skip Download. Datium Dataset found at ../datasets/DatiumSample.rpt


## Read Data

In [4]:
df=pd.read_csv(DATIUM_DATASET_PATH,sep='\t')
df=df.drop(DROP_COLS, axis=1)
nrows, ncols = df.shape
print(f'INFO - Dataset contains {nrows} rows, {ncols} columns')
df.head()

/Users/jerome/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (27,91) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


INFO - Dataset contains 62192 rows, 123 columns


,Make,Model,MakeCode,FamilyCode,YearGroup,MonthGroup,SequenceNum,Description,CurrentRelease,ImportFlag,LimitedEdition,Series,SeriesModelYear,BadgeDescription,BadgeSecondaryDescription,BodyStyleDescription,BodyConfigDescription,WheelBaseConfig,Roofline,ExtraIdentification,DriveDescription,DriveCode,GearTypeDescription,GearLocationDescription,GearNum,DoorNum,EngineSize,EngineDescription,Cylinders,FuelTypeDescription,InductionDescription,OptionCategory,CamDescription,EngineTypeDescription,FuelCapacity,FuelDeliveryDescription,MethodOfDeliveryDescription,GrossCombinationMAss,GrossVehicleMass,VIN,WheelBase,Height,Length,Width,KerbWeight,TareMass,PayLoad,Power,PowerRPMFrom,PowerRPMTo,Torque,TorqueRPMFrom,TorqueRPMTo,RonRating,SeatCapacity,ModelCode,BuildCountryOriginDescription,ValvesCylinder,EngineCycleDescription,EngineConfigurationDescription,EngineLocation,EngineNum,Acceleration,FrontTyreSize,RearTyreSize,FrontRimDesc,RearRimDesc,TowingBrakes,TowingNoBrakes,WarrantyCustAssist,FreeScheduledService,WarrantyYears,WarrantyKM,FirstServiceKM,FirstServiceMonths,RegServiceMonths,AltEngEngineType,AltEngBatteryType,AltEngCurrentType,AltEngAmpHours,AltEngVolts,AltEngChargingMethod,AltEngPower,AltEngPowerFrom,AltEngPowerTo,AltEngTorque,AltEngTorqueFrom,AltEngTorqueTo,AltEngDrive,NormalChargeMins,QuickChargeMins,NormalChargeVoltage,QuickChargeVoltage,KMRangeElectricEng,ElectricEngineLocation,TopSpeedElectricEng,GreenhouseRating,AirpollutionRating,OverallGreenStarRating,CO2Combined,CO2Urban,CO2ExtraUrban,FuelUrban,FuelExtraurban,FuelCombined,EmissionStandard,MaxEthanolBlend,AncapRating,VFactsClass,VFactsSegment,VFactsPrice,IsPPlateApproved,AverageKM,GoodKM,NewPrice,Colour,Branch,SaleCategory,Sold_Date,Compliance_Date,Age_Comp_Months,KM,Sold_Amount
0,Holden,Commodore,HOLD,COMMODO,2008,0,0,VE Omega Sedan 4dr. Auto 4sp 3.6i,F,L,F,VE,NaN,Omega,NaN,Sedan,NaN,NaN,NaN,NaN,Rear Wheel Drive,RWD,Automatic,Floor,4.0,4,3565,3.6,6,Petrol - Unleaded ULP,Aspirated,PASS,DOHC with VVT,Piston,73.0,Multi-Point Injection,Electronic Sequential,NaN,NaN,6G1EK52B#8L######,2915.0,1476.0,4894.0,1899.0,1690.0,1622.0,NaN,180.0,NaN,6000.0,330.0,NaN,2600.0,91.0,5.0,8EK69-114,AUSTRALIA,4.0,4 Stroke,V60,Front,HBA04 ######,NaN,225/60 R16,225/60 R16,16x7.0,16x7.0,2100.0,NaN,3yr Roadside,NaN,3.0,100000.0,15000.0,12.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,3.0,260.0,NaN,NaN,NaN,NaN,10.9,NaN,NaN,4.0,Passenger,Large,< $70K,T,230.0,140.0,34790.0,White,Perth (WA),Auction,2015-11-03 00:00:00.000,02/2008,93.0,227878.0,2000.0
1,Holden,Commodore,HOLD,COMMODO,1993,7,41,VR Executive Wagon 5dr. Auto 4sp 3.8i,F,L,F,VR,NaN,Executive,NaN,Wagon,NaN,NaN,NaN,NaN,Rear Wheel Drive,RWD,Automatic,Floor,4.0,5,3791,3.8,6,Petrol - Unleaded ULP,Aspirated,PASS,Pushrod,Piston,68.0,Multi-Point Injection,Electronic,NaN,NaN,6H8VRK35HPL######,2822.0,1478.0,4903.0,1794.0,1380.0,1380.0,NaN,130.0,NaN,4800.0,295.0,NaN,3200.0,NaN,5.0,NaN,AUSTRALIA,2.0,4 Stroke,V90,Front,VH-######,NaN,205/65 R15,205/65 R15,15x6.0,15x6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.6,9.2,NaN,NaN,NaN,Passenger,Large,< $70K,T,600.0,360.0,27978.0,Red,Belmore (NSW),Auction,2000-10-18 00:00:00.000,08/1993,86.0,153091.0,6800.0
2,Toyota,RAV4,TOYO,RAV4,2012,0,6,ACA33R MY12 CV Wagon 5dr Man 5sp 4x4 2.4i,F,L,F,ACA33R,MY12,CV,NaN,Wagon,NaN,NaN,NaN,NaN,4X4 On Demand,4XO,Manual,Floor,5.0,5,2362,2.4,4,Petrol - Unleaded ULP,Aspirated,SUV,DOHC with VVT,Piston,60.0,Multi-Point Injection,Electronic Sequential,3560.0,2060.0,JTMBD33V*05######,2660.0,1695.0,4625.0,1815.0,1545.0,1510.0,515.0,125.0,NaN,6000.0,224.0,NaN,4000.0,91.0,5.0,ACA33R-ANMXKQ,JAPAN,4.0,4 Stroke,In-line,Front,2AZ-#######,NaN,225/65 R17,225/65 R17,17x6.5,17x6.5,1500.0,750.0,NaN,NaN,3.0,100000.0,5000.0,3.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,6.5,3.5,213.0,NaN,NaN,NaN,NaN,9.1,NaN,Na

## Remove Rows with Null Targets, New Price, KM

In [5]:
def remove_null_rows(df, col):
    target_nulls = df[col].isna()
    if target_nulls.sum():
        print(f'WARNING - Found {target_nulls.sum()} Null Values in : {col}. Will drop rows')
        df = df[~target_nulls]
        nrows, ncols = df.shape
        print(f'INFO - Dataset contains {nrows} rows, {ncols} columns')   
    return df


df=remove_null_rows(df, TARGET_COL)
df=remove_null_rows(df, 'NewPrice')

WARNING - Found 4 Null Values in : Sold_Amount. Will drop rows
INFO - Dataset contains 62188 rows, 123 columns
WARNING - Found 1 Null Values in : NewPrice. Will drop rows
INFO - Dataset contains 62187 rows, 123 columns


## Check Columns with Mixed D_Types

In [6]:
check_cols=[27, 91]
for c in check_cols:
    check_col=df.columns[c]
    print(check_col)
    print('D_Types are: ',set(type(s) for s in df[check_col]))

EngineDescription
D_Types are:  {<class 'str'>, <class 'float'>}
NormalChargeVoltage
D_Types are:  {<class 'str'>, <class 'float'>}


In [7]:
print('INFO - EngineDescription is just a rounded up number for EngineSize')
df[['EngineDescription','EngineSize']]
## ***Add Round Up Verification

INFO - EngineDescription is just a rounded up number for EngineSize


,EngineDescription,EngineSize
0,3.6,3565
1,3.8,3791
2,2.4,2362
3,3.6,3565
4,2.4,2362
...,...,...
62186,3.2,3198
62187,4.0,3984
62188,4.0,3984
62189,2.0,1998


In [8]:
print('INFO - Drop EngineDescription')
df=df.drop('EngineDescription', axis=1)
nrows, ncols = df.shape
print(f'INFO - Dataset contains {nrows} rows, {ncols} columns')

INFO - Drop EngineDescription
INFO - Dataset contains 62187 rows, 122 columns


In [9]:
nulls_pct = df['NormalChargeVoltage'].isna().sum()/nrows*100
print(f'INFO - {round(nulls_pct,3)}% Nulls in NormalChargeVoltage')
print('INFO - Drop NormalChargeVoltage')

df=df.drop('NormalChargeVoltage', axis=1)
nrows, ncols = df.shape
print(f'INFO - Dataset contains {nrows} rows, {ncols} columns')

INFO - 99.997% Nulls in NormalChargeVoltage
INFO - Drop NormalChargeVoltage
INFO - Dataset contains 62187 rows, 121 columns


## Drop Columns with many Nulls

In [10]:
null_cols=df.isnull().sum(axis=0)
null_cols=null_cols[null_cols>0]

In [11]:
print(f'INFO - Dropping Columns with Nulls Rows Larger than Cutoff {CUTOFF_NULL_PCT}%')
drop_cols = null_cols[null_cols> nrows * CUTOFF_NULL_PCT / 100]
print(f'INFO - Dropping {len(drop_cols)}/{len(df.columns)} Numerical Columns due to Nulls')
print(drop_cols)

INFO - Dropping Columns with Nulls Rows Larger than Cutoff 25%
INFO - Dropping 47/121 Numerical Columns due to Nulls
SeriesModelYear              40982
BadgeSecondaryDescription    58423
BodyConfigDescription        51073
WheelBaseConfig              59995
Roofline                     61467
ExtraIdentification          57992
GrossCombinationMAss         31898
GrossVehicleMass             23570
PayLoad                      29194
PowerRPMFrom                 61638
TorqueRPMFrom                54486
Acceleration                 44718
WarrantyCustAssist           33898
FreeScheduledService         62014
FirstServiceKM               21729
FirstServiceMonths           25348
RegServiceMonths             20278
AltEngEngineType             61051
AltEngBatteryType            61052
AltEngCurrentType            61053
AltEngAmpHours               62181
AltEngVolts                  61051
AltEngChargingMethod         61054
AltEngPower                  61054
AltEngPowerFrom              62029
AltEngPo

# End of Preprocess 1

In [12]:
df.to_csv(OUTPUT_DATASET_PATH, index=None)
print(f'INFO - End of Preprocess 1. Saved dataset to {OUTPUT_DATASET_PATH}')

INFO - End of Preprocess 1. Saved dataset to ../datasets/dataset_1.csv
